### Goal: Given a set of text movie reviews that have been labeled negative or positive

For more information on this dataset visit http://ai.stanford.edu/~amaas/data/sentiment/


https://huggingface.co/transformers/v3.2.0/custom_datasets.html

In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("/content/moviereviews.csv",error_bad_lines=False, engine="python")
data.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [5]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.8 MB/s eta 0:00:00


In [6]:
data.isna().sum()

label      0
review    35
dtype: int64

In [7]:
data = data.dropna()

In [8]:
data = data[~data["review"].str.isspace()]

In [9]:
data["toxic"] = pd.get_dummies(data["label"], drop_first=True)

In [10]:
data.head()

,label,review,toxic
0,neg,how do films like mouse hunt get into theatres...,0
1,neg,some talented actresses are blessed with a dem...,0
2,pos,this has been an extraordinary year for austra...,1
3,pos,according to hollywood movies made in last few...,1
4,neg,my first press screening of 1998 and already i...,0


In [11]:
data = data.drop("label",  axis = 1)

In [12]:
import torch

In [13]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [14]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
X = list(data["review"])
y = list(data["toxic"])

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

"""The stratify parameter is used for stratified sampling, which is a technique that ensures 
that each class in the target variable y is represented in both the training and validation sets. 
This is useful when the target variable has an imbalanced distribution, as it helps prevent bias in the model."""

'The stratify parameter is used for stratified sampling, which is a technique that ensures \nthat each class in the target variable y is represented in both the training and validation sets. \nThis is useful when the target variable has an imbalanced distribution, as it helps prevent bias in the model.'

In [18]:
X_train_tokenized = tokenizer(X_train, padding = True, truncation = True, max_length= 512)
X_val_tokenized = tokenizer(X_val, padding= True, truncation= True, max_length= 512)

In [19]:
sample_txt = ["I love apples. Summer is great."]

In [20]:
txt_tokenized = tokenizer(sample_txt, padding = True, truncation = True, max_length = 512)

In [21]:
txt_tokenized

{'input_ids': [[101, 1045, 2293, 18108, 1012, 2621, 2003, 2307, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [22]:
txt_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

# Fine tuning model on our custom dataset

**We need to convert our tokenized output into the dataset format.**

https://huggingface.co/transformers/v3.1.0/custom_datasets.html

In [24]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        # it takes the tokenized output we have and the corresponding labels
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
        # returns the length of our input_ids

This code defines a custom dataset class in PyTorch. Here's a breakdown of what the different parts of the code do:

- The Dataset class is defined as a subclass of torch.utils.data.Dataset. This is a PyTorch built-in class for representing a dataset.
- The __init__ method is the constructor for the Dataset class. It takes two - -arguments: encodings and labels. encodings is a dictionary containing the input encodings for the dataset, and labels is an optional list of labels for the dataset.
- The __getitem__ method is used to retrieve a single item from the dataset. It takes an index idx as input and returns a dictionary containing the input encodings for that index. If labels are present, the label for that index is also included in the dictionary. Note that the input encodings are converted to PyTorch tensors using the torch.tensor method.
- The __len__ method returns the length of the dataset, which is the length of the input_ids in the encodings dictionary.

This custom dataset class can be used in combination with a PyTorch dataloader to create a data iterator for training a model. The dataloader will iterate over the dataset and return batches of data, which can be fed into the model during training.

In [25]:
# lets train our dataset by passing our tokenized training and validation and get dataset object 
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [26]:
# we can apply the __getitem__ method and pass the index and it will return dictionary representaion of our training data
# train_dataset[5]

# Evaluation

In [27]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [28]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Defining Trainer

Trainer is the interface that HuggingFace has created which takes our model which we loaded earlier (model = BertForSequenceClassification.f.........)

In [29]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1550
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 194
  Number of trainable parameters = 109483778


Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
np.set_printoptions(suppress=True) # supresses the values and outputs readable values

In [ ]:
text = "That was good point"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs) # unpacking the input as tokenizer output is a dictionary
print(outputs) # the output is unnormalized
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1) # appliying our softmax to last dimension
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:
trainer.save_model('CustomModel')

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2

In [ ]:
# text = "That was good point"
text = "This was the worst"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions